In [ ]:
import sys
!{sys.executable} -m pip install contractions
!pip install datasets
!pip install evaluate
!pip install fsspec==2023.9.2

!pip install spacy
!python -m spacy download es_core_news_md

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 10.2 MB/s eta 0:00:00



⚙️ **Requerimientos importantes sobre el ejercicio**

- El notebook debe ejecutarse **de principio a fin sin intervención manual**.
- Si utilizas librerías que no están incluidas por defecto en Google Colab, **asegúrate de instalarlas dentro del notebook** (por ejemplo: `!pip install ...`).

- Algunas celdas incluyen identificadores especiales que indican ciertas normas que **debes** respetar:
 - `#NO-MODIFY: DATA LOAD`  
    🔒 **No modifiques** el contenido de esta celda.

  - `#NO-MODIFY: VARIABLE NAME`  
    ✏️ Puedes modificar o añadir información **dentro de la celda**, pero **sin cambiar el nombre de la variable asignada**. No incluyas más variables de las existentes en la celda.

  - `#MODIFY: ADD INFO TO SOLVE FUNCTION`  
    🔧 Puedes modificar el **interior de la función** para resolver la tarea, pero **no cambies su nombre, la cabecera ni el `return`**.



## Imports

In [ ]:
import numpy as np
import nltk
nltk.download('stopwords')

In [ ]:
# Add your imports here
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import string
import re
import contractions
from tqdm.autonotebook import tqdm
tqdm.pandas()

# 🔍 Ejercicio1: Detección de profesiones en tweets

## Enunciado

En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Utilizaremos un subconjunto de los datos de la tarea 1 del shared task [**ProfNER**](https://temu.bsc.es/smm4h-spanish), centrada en la detección de menciones a profesiones en tweets publicados durante la pandemia del COVID-19. El objetivo original de la tarea era analizar que profesiones podrían haber sido especialmente vulnerables en el contexto de la crisis sanitaria.

Para simplificar el ejercicio, he preparado una versión reducida del dataset original. Tu tarea será entrenar un clasificador binario basado en la arquitectura Transformers, que, dado un tweet, determine si contiene una mención explícita a una profesión (etiqueta `1`) o no (etiqueta `0`).




✅ **Objetivos del ejercicio**

A lo largo de este notebook, completarás las siguientes etapas para construir un clasificador de menciones a profesiones en tweets:

1. **Análisis Exploratorio de Datos (EDA)**: Calcular estadísticas básicas del conjunto de datos (como el número de ejemplos del training set, la distribución de clases del dataset, la longitud media de los textos) o crear visualizaciones para cmprender mejor el contenido de los documentos usando wordclouds o histogramas.

2. **Selección y justificación del modelo**: Elegir un modelo del Hub de Huggingface adecuado para los datos con los que se va a trabajar y el tipo de tarea a desarrollar.

3. **Entrenamiento del clasificador**: Entrenar el modelo de forma reproducible y evaluar su rendimiento sobreel conjunto de datos de validación, incluyendo un classification score y matriz de confusion

4. **Generación de predicciones sobre el conjunte de test**: Aplicar el modelo entrenado al conjunto de test, y guardar las predicciones en un archivo `.tsv` de 2 columnas `id` y `label` separadas por tabulador

📝 **Criterios de Evaluación**

Tu trabajo será evaluado según los siguientes criterios:

| Criterio                                            | Peso  |
|-----------------------------------------------------|--------|
| 🔍 Análisis exploratorio y preprocesamiento         | 20%   |
| 🤖 Selección y justificación del modelo             | 25%   |
| 📁 Formato y validez del archivo de predicciones    | 5%    |
| ⚙️ Ejecución correcta del notebook (sin intervención) | 10%   |
| 📈 Rendimiento del modelo sobre el conjunto de test | 30%   |
| ✍️ Claridad y calidad de las explicaciones          | 10%   |



🔔 **Nota importante:**

> El rendimiento del modelo se evaluará utilizando métricas estándar como el **F1-score** sobre el conjunto de test.

> El archivo de predicciones debe respetar **estrictamente** el formato solicitado (`id` y `label`, separados por tabulador y con extensión `.tsv`).  
  ❗ Si el archivo no cumple con este formato, **el ejercicio no podrá ser evaluado en esa sección**.

> El/la estudiante con el **mayor F1-score** obtendrá la puntuación máxima en el apartado de rendimiento. El resto de calificaciones se ajustarán de forma proporcional al mejor resultado



⚙️ **Requerimientos y reglas**

- El notebook debe ejecutarse **de principio a fin sin intervención manual**.
- Si utilizas librerías que no están incluidas por defecto en Google Colab, **asegúrate de instalarlas dentro del notebook** (por ejemplo: `!pip install ...`).

- Algunas celdas incluyen identificadores especiales que indican ciertas normas que **debes** respetar:
 - `#NO-MODIFY: DATA LOAD`  
    🔒 **No modifiques** el contenido de esta celda.

  - `#NO-MODIFY: VARIABLE NAME`  
    ✏️ Puedes modificar o añadir información **dentro de la celda**, pero **sin cambiar el nombre de la variable asignada**. No incluyas más variables de las existentes en la celda.

  - `#MODIFY: ADD INFO TO SOLVE FUNCTION`  
    🔧 Puedes modificar el **interior de la función** para resolver la tarea, pero **no cambies su nombre, la cabecera ni el `return`**.


# Tu resolución (rellena las celdas marcadas)

## Obtención de datos

Descargamos los datos del [repositorio de Huggingface](https://huggingface.co/datasets/luisgasco/profner_classification_master).

In [ ]:
#NO-MODIFY: DATA LOAD
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel
dataset = load_dataset("luisgasco/profner_classification_master")

El dataset contiene tres subsets:
- **train** y **validation**: Contienen el identificador del tweet, el texto, y su etiqueta, que podrá tener valor 1, si contiene una mención de una profesión; o valor 0, si no contiene una mención de una profesión.
- **test**: El test set tambiíen contiene la información de label por un requerimiento de Huggingface, pero el contenido de esta variable es siempre "-1". Es decir que deberéis predecir nuevas etiquetas una vez hayáis entrenado el modelo utilizando el train y el validation set.

## Análisis exploratorio de datos

Para hacer el análisis exploratorio de datos, transformamos cada subset a un pandas dataframe para mayor comodidad.

In [ ]:
#NO-MODIFY: DATA LOAD
dataset_train_df = dataset["train"].to_pandas()
dataset_val_df = dataset["validation"].to_pandas()
dataset_test_df = dataset["test"].to_pandas()

**Número de documentos**

Obten con la función `get_num_docs_evaluation()` el número de documentos del dataset de training y validation.

> Recuerda incorporar la información para el cálculo dentro del a siguiente celda, sin modificar los atributos de entrada ni de salida de la función, ni su nombre.

In [ ]:
dataset_train_df.to_csv('train.csv', index=False)
dataset_val_df.head()
# dataset_test_df.head()
#dataset_train_df[dataset_train_df['tweet_id'] == '1289869622187945984']

In [ ]:
#MODIFY: ADD INFO TO SOLVE FUNCTION
def get_num_docs_evaluation(dataset_df):
  # Modifica la función.
  num_docs = len(dataset_df)

  # Si se quiere obtener únicamente los que tienen textos distintos de vacío
  # num_docs = len(dataset_df[~dataset_df['text'].isna()])

  # No modifiques el return
  return num_docs


Una vez generada la función, puedes utilizarla posteriormente para calcular resultados y comentarlos

In [ ]:
# Aplica la función
print(f"El numero de documentos en TRAIN son: {get_num_docs_evaluation(dataset_train_df)}")
print(f"El numero de documentos en VALIDATION son: {get_num_docs_evaluation(dataset_val_df)}")


**Número de documentos duplicados**

Obten con la función `detect_duplicates_evaluation()` el número de documentos duplicados del dataset de training y validation.

> Recuerda incorporar la información para el cálculo dentro del a siguiente celda, sin modificar los atributos de entrada ni de salida de la función, ni su nombre.

In [ ]:
#MODIFY: ADD INFO TO SOLVE FUNCTION
def detect_duplicates_evaluation(dataset_df):
  # Modifica la función.

  num_duplicates = dataset_df.duplicated(subset='text').sum()
  # No modifiques el return
  return num_duplicates

Una vez generada la función, puedes utilizarla posteriormente para calcular resultados y comentarlos

In [ ]:
# Aplica la función
print(f"El número de documentos duplicados en TRAIN son: {detect_duplicates_evaluation(dataset_train_df)}")
print(f"El número de documentos duplicados en VALIDATION son: {detect_duplicates_evaluation(dataset_val_df)}")

**Número de documentos por cada clase:**


Obten con la función `analyse_num_labels_evaluation()` para calcular el número de documentos de cada categoría en el dataset

> Recuerda incorporar la información para el cálculo dentro del a siguiente celda, sin modificar los atributos de entrada ni de salida de la función, ni su nombre.

In [ ]:
#MODIFY: ADD INFO TO SOLVE FUNCTION
def analyse_num_labels_evaluation(dataset_df):
  # Modifica la función.
  num_positives = len(dataset_df[dataset_df['label'] == 1])
  num_negatives = len(dataset_df[dataset_df['label'] == 0])

  # No modifiques el return
  return num_positives, num_negatives

Una vez generada la función, puedes utilizarla posteriormente para calcular resultados y comentarlos

In [ ]:
# Aplica la función
num_train_positives, num_train_negatives = analyse_num_labels_evaluation(dataset_train_df)
num_val_positives, num_val_negatives = analyse_num_labels_evaluation(dataset_val_df)

print(f"El número de documentos en TRAIN que son positivos son: {num_train_positives} y negativos son: {num_train_negatives}")
print(f"El número de documentos en VALIDATION que son positivos son: {num_val_positives} y negativos son: {num_val_negatives}")


**Distribución de la longitud de los tweet en caracteres:**

In [ ]:
dataset_train_df['char_len'] = dataset_train_df['text'].apply(lambda x: len(x))

# Importamos las librerías matplotlib y seaborn:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(14,12))
sns.set_style("darkgrid")

sns.displot(data=dataset_train_df, x="char_len", hue="label", kde=True, legend=True)
plt.legend(["Incluye profesiones", "No incl. profesiones"])
# Definimos el título de los ejes:
plt.xlabel('Caracteres-TRAIN', fontsize=16)
plt.ylabel('Densidad', fontsize=16)

# Finalmente mostramos el gráfico:
plt.show()

In [ ]:
dataset_train_df[dataset_train_df['label'] == 1].describe()

In [ ]:
dataset_train_df[dataset_train_df['label'] == 0].describe()

> Aquí se observa que los tweets que contienen profesiones tienen a ser más largos que los que no contienen. En el caso de los que si contientienen profesiones, el son en promedio más grandes que los que no los contienen en un 22.36% de longuitud. El tweet mas largo es de 454 (Con profesión) y el mas corto es 10 caracteres(sin profesión). En general los textos donde se mencionan profesiones son mucho más grandes que los que no los incluyen.

In [ ]:
dataset_val_df['char_len'] = dataset_val_df['text'].apply(lambda x: len(x))

# Importamos las librerías matplotlib y seaborn:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(14,12))
sns.set_style("darkgrid")

sns.displot(data=dataset_val_df, x="char_len", hue="label", kde=True, legend=True)
plt.legend(["Incluye profesiones", "No incl. profesiones"])
# Definimos el título de los ejes:
plt.xlabel('Caracteres-VAL', fontsize=16)
plt.ylabel('Densidad', fontsize=16)

# Finalmente mostramos el gráfico:
plt.show()

In [ ]:
dataset_val_df.describe()

**Análisis de contenido de los tweets**

Para ello utiliza wordclouds

In [ ]:
from wordcloud import WordCloud
txt_cat0 = ",".join(dataset_train_df[dataset_train_df['label'] == 0]['text'].to_list())
txt_cat1 = ",".join(dataset_train_df[dataset_train_df['label'] == 1]['text'].to_list())

from nltk.corpus import stopwords
es_stopwords = set(stopwords.words('spanish'))

wordcloud = WordCloud(background_color='white',
                      max_words=5000,
                      contour_width=0,
                      contour_color='plasma',
                      stopwords=es_stopwords,
                      normalize_plurals=True)

wordcloud.generate(txt_cat0)
wordcloud.to_image()

In [ ]:
wordcloud.generate(txt_cat1)
wordcloud.to_image()

> Se observar palabras clave como Coronavirus, Gobierno, Pandemia. Pero hay un gran presencia de textos a referencias. Después de la tokenización y limpieza se podría observar mejor lo que indica. Sin embargo es claro que en el caso donde no se meciona las profesiones, la idea clara es la presencia del virus y el confinaimiento. Mientras que en el caso de donde se menciona la profesion, la palabra sanitario, pandemia y gobierno está representadas con mayor tamaño.

In [ ]:
# SOLO PARA TRAINING
# Utilizando el procedimiento que vimos en clase para limpiar mejor los datasets
# y observar mejor el contenido - Solo para analisis exploratorio

# Eliminar espacios
def eliminar_espacios(text):
    return  " ".join(text.split())

# To lower
def texto_to_lower(text):
  return text.lower()

def replace_contraction(text):
  return contractions.fix(text, slang=True)

# Tokenizador
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

def normalizar_tokens(documento):
    # Dividir el documento en palabras
    palabras =  tweet_tokenizer.tokenize(documento)

    # Reemplazar URLs, menciones de Twitter y números por los tokens correspondientes
    for i in range(len(palabras)):
        if palabras[i].startswith("http://") or palabras[i].startswith("https://") or palabras[i].startswith("www."):
            palabras[i] = "URL"
        elif palabras[i].startswith("@"):
            palabras[i] = "MENTION"
        elif palabras[i].isdigit():
            palabras[i] = "NUM"

    # Unir las palabras de nuevo en un documento modificado
    documento_modificado = ' '.join(palabras)

    return documento_modificado

# Lematizar
import spacy
nlp = spacy.load('es_core_news_md', disable=['parser', 'ner'])
# es_core_news_md -> idioma español

def lematizar_eliminacion_tokens(texto):
    # Procesar el texto con el objeto nlp
    doc = nlp(texto)

    # Lematizar el texto
    lemas = [token.lemma_ for token in doc]

    # Eliminar símbolos de puntuación y stopwords
    tokens_filtrados = [token for token in lemas if token.isalpha() and token.lower() not in es_stopwords]

    # Unir los tokens filtrados en un nuevo texto
    texto_procesado = ' '.join(tokens_filtrados)

    return texto_procesado


In [ ]:
dataset_train_df['normalized'] = dataset_train_df['text'].progress_apply(lambda x: eliminar_espacios(x))
dataset_train_df['normalized'] = dataset_train_df['normalized'].progress_apply(lambda x: texto_to_lower(x))
dataset_train_df['normalized'] = dataset_train_df['normalized'].progress_apply(lambda x: replace_contraction(x))
dataset_train_df['normalized'] = dataset_train_df['normalized'].progress_apply(lambda x: normalizar_tokens(x))
dataset_train_df['normalized'] = dataset_train_df['normalized'].progress_apply(lambda x: lematizar_eliminacion_tokens(x))

In [ ]:
txt_cat_norm0 = ",".join(dataset_train_df[dataset_train_df['label'] == 0]['normalized'].to_list())
txt_cat_norm1 = ",".join(dataset_train_df[dataset_train_df['label'] == 1]['normalized'].to_list())

# Para eliminar el ruido creadas en la depuración
es_stopwords.update(["URL", "MENTION", "NUM"])

wordcloud.generate(txt_cat_norm0)
wordcloud.to_image()

In [ ]:
wordcloud.generate(txt_cat_norm1)
wordcloud.to_image()

## Tokenización

El texto del dataset no está preparado para ser introducido en un modelo Transformers. Lleva a cabo el proceso de tokenización.

In [ ]:
# IMPORTS



In [ ]:
dataset_test_df.head()

Selecciona un modelo apropiado para la tarea:

> Recuerda que en la siguiente celda sólo debes asignar un valor a model_name. No añadas más información en la celda.

In [ ]:
#NO-MODIFY: VARIABLE NAME
model_name = 'FacebookAI/xlm-roberta-base'

> El modelo seleccionado es el FacebookAI/xlm-roberta-base.
Este modelo tiene una gran cantidad des descargas (13,501,854 en el último mes), es multilenguage (por lo que español está incluido), es creado por Facebook (una companía de importancia en la industria), y tiene mucha informacion de soporte.
Fuente:
https://huggingface.co/FacebookAI/xlm-roberta-base

Puedes continuar con el proceso aquí:

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel
dataset_train_hf = Dataset.from_pandas(dataset_train_df).rename_column("label", "labels")
dataset_val_hf = Dataset.from_pandas(dataset_val_df).rename_column("label", "labels")
dataset_test_hf = Dataset.from_pandas(dataset_test_df).rename_column("label", "labels")



In [ ]:
dataset_train_hf = dataset_train_hf.cast_column("labels", ClassLabel(num_classes=2)) # 0,1

In [ ]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    # Tokenizar el texto
    return tokenizer(examples['text'], padding="max_length", truncation=True)

dataset_train_hf = dataset_train_hf.map(preprocess_function, batched=True)
dataset_val_hf = dataset_val_hf.map(preprocess_function, batched=True)
dataset_test_hf = dataset_test_hf.map(preprocess_function, batched=True)

## Fine-tuning

Carga el model para ser ajustado posteriormente:

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "NO_INCL_PROFESION", 1: "INCL_PROFESION"}
label2id = {"NO_INCL_PROFESION": 0, "INCL_PROFESION": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2, id2label=id2label, label2id=label2id)


### Configuracion training_args

Configura los parámetros de entrenamiento del modelo.


>

> Recuerda que en la siguiente celda sólo debes asignar atributos a la variable training_args. No añadas  otras variables en la celda

In [ ]:
#NO-MODIFY: VARIABLE NAME
training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.1,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    logging_strategy="steps",
    logging_steps=50,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
    seed=52
)

### Métricas de evaluación

Define las métricas de evaluación

In [ ]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)["f1"]

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }


### Ajuste del modelo

Lleva a cabo el ajuste del modelo:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_hf,
    eval_dataset=dataset_val_hf,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2) # Podriamoshacer une arly stop
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("output_model_save")

In [ ]:
qwerqwer

## Evaluacion

Una vez llevada a cabo el entrenamiento, realiza la evaluación del modelo.

In [ ]:
trainer.evaluate(dataset_test_hf)

## Genera predicciones

Genera predicciones sobre el test set. Recuerda que el archivo que generes y adjuntes al ejercicio debe tener dos columnas:


| id         | label |
|------------|-------|
| 1234567890 | 1     |
| 1234567891 | 0     |
| 1234567892 | 0     |
| 1234567893 | 1     |

- El archivo debe estar en formato **TSV** (separado por tabuladores).
- Debe contener exactamente **dos columnas**: `id` y `label`.
- Es obligatorio incluir la **cabecera**.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Cambia esto por tu modelo
hub_model_id = "luisgasco/modelo_test"

model = AutoModelForSequenceClassification.from_pretrained(hub_model_id)
tokenizer = AutoTokenizer.from_pretrained(hub_model_id)

dataset_tokenized = dataset.map(preprocess_function, batched=True)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=32,
    do_train=False,
    do_eval=True,
    logging_dir="./logs",
    report_to="none"

#
  )
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_tokenized["validation"],
)